In [1]:
import glob
import pandas as pd
import json
import requests

# Importing and Loading the datasets

In [2]:
df = pd.concat([pd.read_csv(f) for f in glob.glob("./data/*.csv")], ignore_index=True)
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN


In [3]:
df.shape

(911244, 11)

# Handling Null Values

In [4]:
df.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
resale_price                0
remaining_lease        709050
dtype: int64

In [5]:
# droping remaining_lease missing values
df = df.dropna()
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

# MRT (Mass Rapid Transit System) Railway Transportation
*   Getting MRT coordinates for each city
*   so that we can calculate the distance afterwards using MRT coordinates



In [6]:
df['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
       'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN'], dtype=object)

In [7]:
list_of_mrt = [
    'Jurong East MRT Station',
    'Bukit Batok MRT Station',
    'Bukit Gombak MRT Station',
    'Choa Chu Kang MRT Station',
    'Yew Tee MRT Station',
    'Kranji MRT Station',
    'Marsiling MRT Station',
    'Woodlands MRT Station',
    'Admiralty MRT Station',
    'Sembawang MRT Station',
    'Canberra MRT Station',
    'Yishun MRT Station',
    'Khatib MRT Station',
    'Yio Chu Kang MRT Station',
    'Ang Mo Kio MRT Station',
    'Bishan MRT Station',
    'Braddell MRT Station',
    'Toa Payoh MRT Station',
    'Novena MRT Station',
    'Newton MRT Station',
    'Orchard MRT Station',
    'Somerset MRT Station',
    'Dhoby Ghaut MRT Station',
    'City Hall MRT Station',
    'Raffles Place MRT Station',
    'Marina Bay MRT Station',
    'Marina South Pier MRT Station',
    'Pasir Ris MRT Station',
    'Tampines MRT Station',
    'Simei MRT Station',
    'Tanah Merah MRT Station',
    'Bedok MRT Station',
    'Kembangan MRT Station',
    'Eunos MRT Station',
    'Paya Lebar MRT Station',
    'Aljunied MRT Station',
    'Kallang MRT Station',
    'Lavender MRT Station',
    'Bugis MRT Station',
    'Tanjong Pagar MRT Station',
    'Outram Park MRT Station',
    'Tiong Bahru MRT Station',
    'Redhill MRT Station',
    'Queenstown MRT Station',
    'Commonwealth MRT Station',
    'Buona Vista MRT Station',
    'Dover MRT Station',
    'Clementi MRT Station',
    'Chinese Garden MRT Station',
    'Lakeside MRT Station',
    'Boon Lay MRT Station',
    'Pioneer MRT Station',
    'Joo Koon MRT Station',
    'Gul Circle MRT Station',
    'Tuas Crescent MRT Station',
    'Tuas West Road MRT Station',
    'Tuas Link MRT Station',
    'Expo MRT Station',
    'Changi Airport MRT Station',
    'HarbourFront MRT Station',
    'Chinatown MRT Station',
    'Clarke Quay MRT Station',
    'Little India MRT Station',
    'Farrer Park MRT Station',
    'Boon Keng MRT Station',
    'Potong Pasir MRT Station',
    'Woodleigh MRT Station',
    'Serangoon MRT Station',
    'Kovan MRT Station',
    'Hougang MRT Station',
    'Buangkok MRT Station',
    'Sengkang MRT Station',
    'Punggol MRT Station',
    'Bras Basah MRT Station',
    'Esplanade MRT Station',
    'Promenade MRT Station',
    'Nicoll Highway MRT Station',
    'Stadium MRT Station',
    'Mountbatten MRT Station',
    'Dakota MRT Station',
    'MacPherson MRT Station',
    'Tai Seng MRT Station',
    'Bartley MRT Station',
    'Lorong Chuan MRT Station',
    'Marymount MRT Station',
    'Caldecott MRT Station',
    'Botanic Gardens MRT Station',
    'Farrer Road MRT Station',
    'Holland Village MRT Station',
    'one-north MRT Station',
    'Kent Ridge MRT Station',
    'Haw Par Villa MRT Station',
    'Pasir Panjang MRT Station',
    'Labrador Park MRT Station',
    'Telok Blangah MRT Station',
    'Bayfront MRT Station',
    'Bukit Panjang MRT Station',
    'Cashew MRT Station',
    'Hillview MRT Station',
    'Beauty World MRT Station',
    'King Albert Park MRT Station',
    'Sixth Avenue MRT Station',
    'Tan Kah Kee MRT Station',
    'Stevens MRT Station',
    'Rochor MRT Station',
    'Downtown MRT Station',
    'Telok Ayer MRT Station',
    'Fort Canning MRT Station',
    'Bencoolen MRT Station',
    'Jalan Besar MRT Station',
    'Bendemeer MRT Station',
    'Geylang Bahru MRT Station',
    'Mattar MRT Station',
    'Ubi MRT Station',
    'Kaki Bukit MRT Station',
    'Bedok North MRT Station',
    'Bedok Reservoir MRT Station',
    'Tampines West MRT Station',
    'Tampines East MRT Station',
    'Upper Changi MRT Station',
    'Woodlands North MRT Station',
    'Woodlands South MRT Station'
]

In [8]:
mrt_lat = []
mrt_long = []

for i in range(0, len(list_of_mrt)):
    query_address = list_of_mrt[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_mrt=json.loads(resp.content)

    if data_mrt['found'] != 0:
        mrt_lat.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        mrt_lat.append('NotFound')
        mrt_lat.append('NotFound')
        print ("No Results")

Jurong East MRT Station,Lat: 1.33329506563598 Long: 103.742153884191
Bukit Batok MRT Station,Lat: 1.3487108105756 Long: 103.749334517338
Bukit Gombak MRT Station,Lat: 1.35861159094192 Long: 103.751790910733
Choa Chu Kang MRT Station,Lat: 1.38536316540225 Long: 103.744370779756
Yew Tee MRT Station,Lat: 1.39757979021386 Long: 103.747459490883
Kranji MRT Station,Lat: 1.42508698073648 Long: 103.762137459497
Marsiling MRT Station,Lat: 1.43252114855026 Long: 103.774074641403
Woodlands MRT Station,Lat: 1.43605761708128 Long: 103.787938777173
Admiralty MRT Station,Lat: 1.44058856161847 Long: 103.800990519771
Sembawang MRT Station,Lat: 1.44905082158502 Long: 103.820046140211
Canberra MRT Station,Lat: 1.44307664075699 Long: 103.829702590959
Yishun MRT Station,Lat: 1.42944308477331 Long: 103.835005047246
Khatib MRT Station,Lat: 1.41738337009565 Long: 103.832979908243
Yio Chu Kang MRT Station,Lat: 1.38168259989517 Long: 103.844991053696
Ang Mo Kio MRT Station,Lat: 1.36993284962264 Long: 103.849558

In [9]:
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'latitude': mrt_lat,
    'longitude': mrt_long
})

# Getting coordinates of each HDB(Housing and Developing Board) Resale flat
in order to conduct the distance from MRT stations as well as

*   So, that we can take out the distance of flats from MRT stations (Mass Rapid Transit System).
*   And, also from CBD (Central Business District).


In [10]:
df['address'] = df['block'] + " " + df['street_name']
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,address
339399,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,232000.0,61 years 04 months,406 ANG MO KIO AVE 10
339400,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,250000.0,60 years 07 months,108 ANG MO KIO AVE 4
339401,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,262000.0,62 years 05 months,602 ANG MO KIO AVE 5
339402,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,265000.0,62 years 01 month,465 ANG MO KIO AVE 10
339403,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,265000.0,62 years 05 months,601 ANG MO KIO AVE 5
...,...,...,...,...,...,...,...,...,...,...,...,...
541588,2016-12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,488000.0,82,297 YISHUN ST 20
541589,2016-12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,455000.0,69,838 YISHUN ST 81
541590,2016-12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,778000.0,74,664 YISHUN AVE 4
541591,2016-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,575000.0,70,325 YISHUN CTRL


In [11]:
address_list = df['address'].unique()

In [12]:
latitude = []
longitude = []
blk_no = []
road_name = []
postal_code = []
address = []
count = 0

for row in range(len(address_list)):
    query_address = address_list[row]
    query_string='https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)


    data_geo_location=json.loads(resp.content)
    if data_geo_location['found'] != 0:
        latitude.append(data_geo_location['results'][0]['LATITUDE'])
        longitude.append(data_geo_location['results'][0]['LONGITUDE'])
        blk_no.append(data_geo_location['results'][0]['BLK_NO'])
        road_name.append(data_geo_location['results'][0]['ROAD_NAME'])
        postal_code.append(data_geo_location['results'][0]['POSTAL'])
        address.append(query_address)
        print (str(query_address) + " ,Lat: " + data_geo_location['results'][0]['LATITUDE'] + " Long: " + data_geo_location['results'][0]['LONGITUDE'])
    else:
        print ("No Results")

Streaming output truncated to the last 5000 lines.
142 LOR 2 TOA PAYOH ,Lat: 1.33583774995002 Long: 103.845162997311
141 POTONG PASIR AVE 3 ,Lat: 1.33405922561445 Long: 103.867455100715
239 LOR 1 TOA PAYOH ,Lat: 1.34050894159915 Long: 103.851040712112
135 POTONG PASIR AVE 3 ,Lat: 1.33469058687886 Long: 103.866824291288
10 LOR 7 TOA PAYOH ,Lat: 1.33716527577081 Long: 103.857228807914
79A TOA PAYOH CTRL ,Lat: 1.33453129537675 Long: 103.847565695822
10 JOO SENG RD ,Lat: 1.33461833377097 Long: 103.88016502252
97 LOR 3 TOA PAYOH ,Lat: 1.33862553964099 Long: 103.847769284458
138B LOR 1A TOA PAYOH ,Lat: 1.33626408939965 Long: 103.845626191384
136 POTONG PASIR AVE 3 ,Lat: 1.33398254838691 Long: 103.866520142842
145 POTONG PASIR AVE 2 ,Lat: 1.33237317770639 Long: 103.866165158665
19 MARSILING LANE ,Lat: 1.44328204285332 Long: 103.777294228046
166 WOODLANDS ST 13 ,Lat: 1.43363334048141 Long: 103.775400477732
206 MARSILING DR ,Lat: 1.44540729578602 Long: 103.77336982144
218 MARSILING CRES ,Lat: 1

In [13]:
df_coordinates = pd.DataFrame({
    'latitude': latitude,
    'longitude': longitude,
    'blk_no': blk_no,
    'road_name': road_name,
    'postal_code': postal_code,
    'address': address
})
len(df_coordinates)

9531

In [14]:
list_of_lat = df_coordinates['latitude']
list_of_long = df_coordinates['longitude']
mrt_lat = mrt_location['latitude']
mrt_long = mrt_location['longitude']

In [15]:
list_of_coordinates = []
list_of_mrt_coordinates = []

for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(mrt_lat, mrt_long):
    list_of_mrt_coordinates.append((lat, long))

# Getting the diatnce with the help of OpenMap API

In [16]:
# Distance to nearest MRT
from geopy.distance import geodesic

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mrt_coordinates)):
        list_of_dist_mrt.append(geodesic(origin,list_of_mrt_coordinates[destination]).meters)
    shortest = (min(list_of_dist_mrt))
    min_dist_mrt.append(shortest)
    list_of_dist_mrt.clear()

In [17]:
# Distance from CDB
cbd_dist = []

for origin in list_of_coordinates:
    cbd_dist.append(geodesic(origin,(1.2830, 103.8513)).meters) #CBD coordinates

In [18]:
# Put MRT and CBD distance together
df_coordinates['cbd_dist'] = cbd_dist
df_coordinates['min_dist_mrt'] = min_dist_mrt

In [19]:
df_coordinates

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt
0,1.36200453938712,103.853879910407,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595
1,1.37096635222625,103.838201940326,108,ANG MO KIO AVENUE 4,560108,108 ANG MO KIO AVE 4,9835.486576,1268.958162
2,1.38070883044887,103.835368226602,602,ANG MO KIO AVENUE 5,560602,602 ANG MO KIO AVE 5,10948.659905,1076.298560
3,1.3662010408294,103.857200967235,465,ANG MO KIO AVENUE 10,560465,465 ANG MO KIO AVE 10,9223.353648,945.371837
4,1.38104135523576,103.835131742647,601,ANG MO KIO AVENUE 5,560601,601 ANG MO KIO AVE 5,10989.223687,1099.505343
...,...,...,...,...,...,...,...,...
9526,1.35546083614203,103.937118935774,860B,TAMPINES AVENUE 5,522860,860B TAMPINES AVE 5,12466.578115,1097.338326
9527,1.43916567473693,103.779976701121,121,MARSILING RISE,730121,121 MARSILING RISE,19004.920891,950.372898
9528,1.42142183300841,103.83924942213,659,YISHUN AVENUE 4,760659,659 YISHUN AVE 4,15364.619802,828.374098
9529,1.37714832714712,103.774023829674,271,BANGKIT ROAD,670271,271 BANGKIT RD,13503.256987,1339.847514


In [20]:
df_coordinates.to_csv('df_coordinates.csv',index=False)

In [21]:
df_coordinates = pd.read_csv('df_coordinates.csv')

In [22]:
df_new = df_coordinates.merge(df, on="address", how='outer')
df_new

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,232000.0,61 years 04 months
1,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,235000.0,61 years 01 month
2,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,202000.0,60 years 02 months
3,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,210000.0,60 years 02 months
4,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,07 TO 09,44.0,Improved,1979,220000.0,60 years 01 month
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202189,1.355461,103.937119,860B,TAMPINES AVENUE 5,522860,860B TAMPINES AVE 5,12466.578115,1097.338326,2016-09,TAMPINES,4 ROOM,860B,TAMPINES AVE 5,04 TO 06,104.0,Model A,1987,460000.0,70
202190,1.439166,103.779977,121,MARSILING RISE,730121,121 MARSILING RISE,19004.920891,950.372898,2016-10,WOODLANDS,EXECUTIVE,121,MARSILING RISE,04 TO 06,170.0,Apartment,1995,670000.0,77
202191,1.421422,103.839249,659,YISHUN AVENUE 4,760659,659 YISHUN AVE 4,15364.619802,828.374098,2016-10,YISHUN,4 ROOM,659,YISHUN AVE 4,04 TO 06,89.0,Simplified,1987,335000.0,69
202192,1.377148,103.774024,271,BANGKIT ROAD,670271,271 BANGKIT RD,13503.256987,1339.847514,2016-11,BUKIT PANJANG,5 ROOM,271,BANGKIT RD,04 TO 06,122.0,Improved,1988,425000.0,70


In [23]:
df_new['resale_price'] = df_new['resale_price'].astype('float')
df_new['floor_area_sqm'] = df_new['floor_area_sqm'].astype('float')
df_new['lease_commence_date'] = df_new['lease_commence_date'].astype('int64')
df_new['lease_remain_years'] = 99 - (2023 - df_new['lease_commence_date'])

df_new.dropna(inplace=True)

df_new

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,lease_remain_years
0,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,232000.0,61 years 04 months,55
1,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,235000.0,61 years 01 month,55
2,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,202000.0,60 years 02 months,55
3,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,210000.0,60 years 02 months,55
4,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,8740.633473,999.941595,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,07 TO 09,44.0,Improved,1979,220000.0,60 years 01 month,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202189,1.355461,103.937119,860B,TAMPINES AVENUE 5,522860,860B TAMPINES AVE 5,12466.578115,1097.338326,2016-09,TAMPINES,4 ROOM,860B,TAMPINES AVE 5,04 TO 06,104.0,Model A,1987,460000.0,70,63
202190,1.439166,103.779977,121,MARSILING RISE,730121,121 MARSILING RISE,19004.920891,950.372898,2016-10,WOODLANDS,EXECUTIVE,121,MARSILING RISE,04 TO 06,170.0,Apartment,1995,670000.0,77,71
202191,1.421422,103.839249,659,YISHUN AVENUE 4,760659,659 YISHUN AVE 4,15364.619802,828.374098,2016-10,YISHUN,4 ROOM,659,YISHUN AVE 4,04 TO 06,89.0,Simplified,1987,335000.0,69,63
202192,1.377148,103.774024,271,BANGKIT ROAD,670271,271 BANGKIT RD,13503.256987,1339.847514,2016-11,BUKIT PANJANG,5 ROOM,271,BANGKIT RD,04 TO 06,122.0,Improved,1988,425000.0,70,64


# Final Combined Data

In [24]:
df_new.to_csv('combined.csv', index = False)